In [34]:
# merge grooming angles and grooming coords from wt flies (5.22.19, 5.24.19, and 5.27.19)
# manual labels with qualitative grooming observations (height, concentration, 
# left/right bias, etc)

import re
import umap
import os.path
import fnmatch
import warnings
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
from scipy import signal, stats
from matplotlib.patches import Patch
from sklearn.decomposition import PCA
from sklearn.manifold import TSNE

warnings.filterwarnings('ignore')

In [35]:
def adjust_bout_numbers(angles, pose3d, cumulative_bouts):
    
    bout_numbers, index = np.unique(angles.behavior_bout, return_index = True)
    bout_numbers_sorted = bout_numbers[index.argsort()]
    bout_numbers_new = np.arange(cumulative_bouts, cumulative_bouts + len(bout_numbers), 1)
    cumulative_bouts = cumulative_bouts + len(bout_numbers)
    
    for j in range(len(bout_numbers_sorted)):
        
        angles['behavior_bout'].replace({bout_numbers_sorted[j]:bout_numbers_new[j]}, inplace=True)
        pose3d['behavior_bout'].replace({bout_numbers_sorted[j]:bout_numbers_new[j]}, inplace=True)
    
    return angles, pose3d, cumulative_bouts

In [42]:
# load training data (angles, pose3d, behaviors)
prefix = r'/media/turritopsis/katie/behavior-classifier'
angles_path = os.path.join(prefix, 'angles_fictrac_training.csv')
data_path = os.path.join(prefix, 'pose3d_fictrac_training.csv')
behaviors_path = os.path.join(prefix, 'behavior_labels_training.csv')
features_path = os.path.join(prefix, 'behavior_labels_features.csv')

FPS = 300.0 # I happen to know this for this dataset

data = pd.read_csv(data_path)
angles = pd.read_csv(angles_path)
behaviors = pd.read_csv(behaviors_path, index_col = False)
behaviors = behaviors[behaviors.filename.isin(angles.filename)]

In [43]:
angles = pd.read_csv(angles_path, engine = 'python', sep = None)
pose3d = pd.read_csv(data_path, engine = 'python', sep = None)
behaviors = pd.read_csv(behaviors_path, engine = 'python', sep = None)
features = pd.read_csv(features_path, engine = 'python', sep = None)

angles_merged = pd.merge(angles, behaviors, how='outer')
pose3d_merged = pd.merge(pose3d, behaviors, how='outer') 

angles_grooming = angles_merged[angles_merged.behavior == 'T1_grooming']
pose3d_grooming = pose3d_merged[pose3d_merged.behavior == 'T1_grooming']
#angles_grooming.to_csv(os.path.join(path, 'angles_fictrac_grooming_labeled.csv'))
#pose3d_grooming.to_csv(os.path.join(path, 'pose_3d_fictrac_grooming_labeled.csv'))

In [44]:
angles_grooming = angles_grooming[~angles_grooming.isnull()]
angles_grooming = angles_grooming.sort_values(by = 'filename')
pose3d_grooming = pose3d_grooming[~pose3d_grooming.isnull()]
pose3d_grooming = pose3d_grooming.sort_values(by = 'filename')
feature_grooming = features.sort_values(by = 'filename')
angles_grooming, pose3d_grooming, cumulative_bouts = adjust_bout_numbers(angles_grooming, pose3d_grooming, 1)

In [45]:
bout_numbers = np.unique(angles_grooming.behavior_bout)
label_types = ['height', 'concentration', 'motion_bias', 'synchronicity', 'side_bias']
features_df = pd.DataFrame(columns = label_types)
for j in range(len(bout_numbers)):
    
    bout_length = len(angles_grooming[angles_grooming['behavior_bout'] == bout_numbers[j]])
    row = features[features['bout_number'] == bout_numbers[j]][label_types]
    rows = pd.concat([row]*bout_length)
    features_df = features_df.append(rows)

In [46]:
features_df = features_df.reset_index(drop = True)
angles_grooming = angles_grooming.reset_index(drop = True)
pose3d_grooming = pose3d_grooming.reset_index(drop = True)
features_angles = angles_grooming.join(features_df)
features_pose3d = pose3d_grooming.join(features_df)

In [47]:
features_angles.to_csv(os.path.join(prefix, 'angles_fictrac_grooming_features.csv'))
features_pose3d.to_csv(os.path.join(prefix, 'pose_3d_fictrac_grooming_features.csv'))